<a href="https://colab.research.google.com/github/VarunB1234/Depression-Detection/blob/main/videoCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import classification_report

# ------------------ Preprocessing Functions ------------------

def processData(data):
    X = data.iloc[:, :].values
    X = np.delete(X, 0, 1)
    X = np.delete(X, 1, 1)
    for i in range(len(X)):
        if isinstance(X[i][5], str) or isinstance(X[i][7], str):
            X[i] = np.zeros((X.shape[1],))
    return X

def getData(filename):
    data = pd.read_csv(filename, delimiter=',', engine='python')
    X = processData(data)
    return X

def makeDataPoint(patientNo):
    file_base = patientNo
    paths = [f"{file_base}_CLNF_AUs.txt",
             f"{file_base}_CLNF_features.txt",
             f"{file_base}_CLNF_features3D.txt",
             f"{file_base}_CLNF_gaze.txt",
             f"{file_base}_CLNF_pose.txt"]
    arrays = [getData(path) for path in paths]
    return np.concatenate(arrays, axis=1)

def scale_down(X):
    X_new = []
    size = 2
    for i in range(int(X.shape[0] / size)):
        cur_row = X[i * size]
        for j in range(1, size):
            if i + j < X.shape[0]:
                cur_row += X[i + j]
        cur_row = cur_row / size
        X_new.append(cur_row)
    return np.array(X_new)

def decrease_size(X):
    size = 10000
    if X.shape[0] < size:
        padding = np.zeros((size - X.shape[0], X.shape[1]))
        X = np.concatenate((X, padding), axis=0)
    elif X.shape[0] > size:
        X = X[:size, :]
    return X

def makeDataset(location, folder):
    file = np.array(pd.read_csv(location, delimiter=',', encoding='utf-8'))[:, 0:2]
    X_temp = []
    Y_temp = []
    for i in range(len(file)):
        patientID = str(int(file[i][0]))
        file_path = f'/content/drive/My Drive/MCA Dataset/{folder}/{patientID}'
        XT = makeDataPoint(file_path)
        XT = scale_down(XT)
        XT = decrease_size(XT)
        X_temp.append(XT)
        Y_temp.append(int(file[i][1]))
    return np.array(X_temp, dtype=np.float32), np.array(Y_temp)

# ------------------ CNN Video Model ------------------

class CNN_Video:
    def __init__(self):
        self.classifier = Sequential()
        self.classifier.add(Conv1D(300, 3, input_shape=(10000, 388), activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=2))
        self.classifier.add(Conv1D(150, 3, activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=2))
        self.classifier.add(Conv1D(75, 3, activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=2))
        self.classifier.add(Conv1D(32, 3, activation='relu'))
        self.classifier.add(MaxPooling1D(pool_size=2))
        self.classifier.add(Flatten())
        self.classifier.add(Dense(units=128, activation='relu'))
        self.classifier.add(Dense(units=1, activation='sigmoid'))
        self.classifier.compile(optimizer='adam', loss='binary_crossentropy')

    def fitModel(self, Xtrain, Ytrain, epoch=5):
        return self.classifier.fit(Xtrain, Ytrain, epochs=epoch, batch_size=4)

    def predictModel(self, X):
        return self.classifier.predict(X)

# ------------------ Load Data ------------------

# Load training data (train + test)
X_train, Y_train = makeDataset('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv', 'train_data')
X_test, Y_test = makeDataset('/content/drive/My Drive/MCA Dataset/full_test_split.csv', 'test_data')

X_train_total = np.concatenate((X_train, X_test), axis=0)
Y_train_total = np.concatenate((Y_train, Y_test), axis=0)

# Load validation data (dev)
X_val, Y_val = makeDataset('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv', 'dev_data')

# ------------------ Train and Evaluate ------------------

model = CNN_Video()
model.fitModel(X_train_total, Y_train_total, epoch=5)

# Predict on dev set
Y_val_pred = model.predictModel(X_val)
Y_val_pred_class = [1 if p > 0.5 else 0 for p in Y_val_pred]

print(classification_report(Y_val, Y_val_pred_class))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 630ms/step - loss: 85175.2344
Epoch 2/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 22s 557ms/step - loss: 30713.3926
Epoch 3/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 518ms/step - loss: 50492.0664
Epoch 4/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 510ms/step - loss: 17271.0371
Epoch 5/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 516ms/step - loss: 962.1739
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 292ms/step
              precision    recall  f1-score   support

           0       0.83      0.43      0.57        23
           1       0.43      0.83      0.57        12

    accuracy                           0.57        35
   macro avg       0.63      0.63      0.57        35
weighted avg       0.70      0.57      0.57        35

